In [ ]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2



if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    wine_path = "https://raw.githubusercontent.com/databricks/mlflow-example-sklearn-elasticnet-wine/master/wine-quality.csv"
    data = pd.read_csv(wine_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    #alpha = 0.3
    l1_ratio = 0.5
    
    #mlflow.create_experiment("Experimento Wine")
    mlflow.set_experiment("Experimento Wine")
    
    for alpha in np.arange(0.1,1.0,0.05):
        for l1_ratio in np.arange(0.1,1.0,0.05):
            with mlflow.start_run():
                lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
                lr.fit(train_x, train_y)

                predicted_qualities = lr.predict(test_x)

                (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

                print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
                print("  RMSE: %s" % rmse)
                print("  MAE: %s" % mae)
                print("  R2: %s" % r2)

                mlflow.log_param("alpha", alpha)
                mlflow.log_param("l1_ratio", l1_ratio)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)
                print("Going to log model")
                mlflow.sklearn.log_model(lr,"wine")
                print("Done!")


Run this locally to push a Ubuntu image into Amazon ECR.

mlflow sagemaker build-and-push-container

In [ ]:
import mlflow.sagemaker as mfs
app_name = "wine-deployment-goyeah"
mfs.deploy(app_name=app_name, model_uri="models:/deployment-wine/1",
           image_url="301813298158.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:1.7.2",
           region_name="us-east-1", mode=mfs.DEPLOYMENT_MODE_REPLACE)

In [ ]:
import boto3
import json
from sklearn import datasets

app_name = "wine-deployment-goyeah"
wine_path = "https://raw.githubusercontent.com/databricks/mlflow-example-sklearn-elasticnet-wine/master/wine-quality.csv"
data = pd.read_csv(wine_path)
endpoint_name = app_name
print(endpoint_name)
print(boto3.client('sts').get_caller_identity())
runtime = boto3.Session().client(service_name='sagemaker-runtime',region_name='us-east-1')

data = data.drop("quality",axis=1)
input_json = data.iloc[[2]].to_json(orient="split",index=False)
print(input_json)

response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=input_json)
result = json.loads(response['Body'].read().decode("utf-8"))
print("Prediction: {}".format(result))

In [ ]:
!wget https://runtime.sagemaker.eu-east-1.amazonaws.com/endpoints/wine-deployment-goyeah/invocations

In [ ]:
!nslookup runtime.sagemaker.eu-east-1.amazonaws.com